In [9]:
#default_exp datasource

In [10]:
#export
from fastai2.basics import *
from fastai2.vision import *

### set `data.yaml`

In [11]:
import yaml,os

In [12]:
os.environ['YAML_DATA']="/home/turgutluk/Vent_Seg_Project/dev/configs/data.yaml"

In [13]:
with open(os.environ.get('YAML_DATA', '../data.yaml')) as f: data_config = yaml.load(f.read(), yaml.FullLoader)

In [14]:
data_config

{'input': {'ATLAS_PATH': '/data/public/PICARE_BMETS_Raw_DICOM_Files',
  'MR_PATH': '/data/public/PICARE_SEGMENTATION_BRAINVENT_MR_V1',
  'CT_PATH': '/data/public/Training_CT_Raw_DICOM_Files',
  'MR_TEST2_PATH': '/data/public/Testing_MR_Raw_DICOM_Files',
  'CT_TEST2_PATH': '/data/public/Testing_CT_Raw_DICOM_Files'},
 'output': {'ATLAS': '/home/turgutluk/data/ventricles_data/atlas',
  'MR': '/home/turgutluk/data/ventricles_data/mr',
  'CT': '/home/turgutluk/data/ventricles_data/ct'},
 'csv_splits': {'ATLAS': '/home/turgutluk/data/ventricles_data/csvs/atlas_splits_df.csv',
  'MR': '/home/turgutluk/data/ventricles_data/csvs/mr_splits_df.csv',
  'CT': '/home/turgutluk/data/ventricles_data/csvs/ct_splits_df.csv'}}

### [Ignore]

In [4]:
# #export
# data_paths = types.SimpleNamespace(
#     ATLAS_OUTPUT="/home/turgutluk/data/ventricles_data/atlas",
#     MR_OUTPUT="/home/turgutluk/data/ventricles_data/mr",
#     CT_OUTPUT="/home/turgutluk/data/ventricles_data/ct")

# #export
# csv_paths = types.SimpleNamespace(
#     CT_META = '/home/turgutluk/data/ventricles_data/csvs/CT_PATH_META.csv',
#     CT_TEST2_META = '/home/turgutluk/data/ventricles_data/csvs/CT_TEST2_PATH_META.csv',
#     MR_META = '/home/turgutluk/data/ventricles_data/csvs/MR_PATH_META.csv',
#     MR_TEST2_META = '/home/turgutluk/data/ventricles_data/csvs/MR_TEST2_PATH_META.csv'
# )

# #export
# ct_metadf = pd.read_csv(csv_paths.CT_META,low_memory=False)
# ct_test2_metadf = pd.read_csv(csv_paths.CT_TEST2_META,low_memory=False)
# mr_metadf = pd.read_csv(csv_paths.MR_META,low_memory=False)
# mr_test2_metadf = pd.read_csv(csv_paths.MR_TEST2_META,low_memory=False)

# #export
# val_test_patients = types.SimpleNamespace(
# mr_val = {'ANON61382','ANON55375','ANON85534','ANON54218','ANON24182','ANON14135','ANON49037',
#           'ANON66932','ANON10465','ANON39801','ANON14447','ANON42229','ANON99458','ANON36946',
#           'ANON16732'},
# mr_test1 = {'ANON78381','ANON38662','ANON78219','ANON65248','ANON98217','ANON22366', 'ANON53486',
#             'ANON80073','ANON93045','ANON26348','ANON72855','ANON60446','ANON28622','ANON60751',
#             'ANON41567'},
# ct_val = {'ANON85656','ANON24135','ANON45434','ANON53464','ANON50198','ANON86095','ANON47701',
#           'ANON21818','ANON13928','ANON45164','ANON57908','ANON10634','ANON37574','ANON13983',
#           'ANON39193','ANON52842','ANON83901','ANON34509','ANON14150','ANON70712','ANON36668',
#           'ANON86933','ANON69869','ANON55750'},
# ct_test1 = {'ANON95021', 'ANON17272', 'ANON45950', 'ANON71219', 'ANON84614', 'ANON22673',
#             'ANON65837', 'ANON51808', 'ANON24224'})

# #export
# # MR train, val, test1 and test2 StudyInstance UIDs
# mr_val_suids = np.unique(mr_metadf[mr_metadf.PatientID.isin(val_test_patients.mr_val)].StudyInstanceUID)
# mr_test1_suids = np.unique(mr_metadf[mr_metadf.PatientID.isin(val_test_patients.mr_test1)].StudyInstanceUID)
# mr_test2_suids = np.unique(mr_test2_metadf.StudyInstanceUID)
# mr_train_suids = (np.unique(mr_metadf[~mr_metadf.StudyInstanceUID
#                                       .isin(np.concatenate([mr_val_suids, mr_test1_suids]))]
#                                       .StudyInstanceUID))

# len(mr_train_suids), len(mr_val_suids), len(mr_test1_suids), len(mr_test2_suids)

# #export
# # CT train, val, test1 and test2 StudyInstance UIDs
# ct_val_suids = np.unique(ct_metadf[ct_metadf.PatientID.isin(val_test_patients.ct_val)].StudyInstanceUID)
# ct_test1_suids = np.unique(ct_metadf[ct_metadf.PatientID.isin(val_test_patients.ct_test1)].StudyInstanceUID)
# ct_test2_suids = np.unique(ct_test2_metadf.StudyInstanceUID)
# ct_train_suids = (np.unique(ct_metadf[~ct_metadf.StudyInstanceUID
#                                       .isin(np.concatenate([ct_val_suids, ct_test1_suids]))]
#                                       .StudyInstanceUID))

# len(ct_train_suids), len(ct_val_suids), len(ct_test1_suids), len(ct_test2_suids)

### [Ignore] create mr (train, val, test1, test2) metadata

In [5]:
# output_path = data_paths.MR_OUTPUT

# mr_files = get_files(output_path, extensions=['.pt'])

# mr_file_suids = np.unique([o.name.split("_")[0] for o in mr_files])

# mr_splitsdf = pd.DataFrame({"StudyInstanceId":mr_file_suids, "SplitType":None})

# mr_splitsdf.loc[mr_splitsdf.StudyInstanceId.isin(mr_train_suids), "SplitType"] = "train"
# mr_splitsdf.loc[mr_splitsdf.StudyInstanceId.isin(mr_val_suids), "SplitType"] = "valid"
# mr_splitsdf.loc[mr_splitsdf.StudyInstanceId.isin(mr_test1_suids), "SplitType"] = "test1"
# mr_splitsdf.loc[mr_splitsdf.StudyInstanceId.isin(mr_test2_suids), "SplitType"] = "test2"

# mr_splitsdf.SplitType.value_counts()

# mr_splitsdf.head()

# mr_splitsdf.to_csv("/home/turgutluk/data/ventricles_data/csvs/mr_splits_df.csv", index=False)

### [Ignore] create ct (train, val, test1, test2) metadata

In [6]:
# output_path = data_paths.CT_OUTPUT

# ct_files = get_files(output_path, extensions=['.pt'])

# ct_file_suids = np.unique([o.name.split("_")[0] for o in ct_files])

# ct_splitsdf = pd.DataFrame({"StudyInstanceId":ct_file_suids, "SplitType":None})

# ct_splitsdf.loc[ct_splitsdf.StudyInstanceId.isin(ct_train_suids), "SplitType"] = "train"
# ct_splitsdf.loc[ct_splitsdf.StudyInstanceId.isin(ct_val_suids), "SplitType"] = "valid"
# ct_splitsdf.loc[ct_splitsdf.StudyInstanceId.isin(ct_test1_suids), "SplitType"] = "test1"
# ct_splitsdf.loc[ct_splitsdf.StudyInstanceId.isin(ct_test2_suids), "SplitType"] = "test2"

# ct_splitsdf.SplitType.value_counts()

# ct_splitsdf.head()

# ct_splitsdf.to_csv("/home/turgutluk/data/ventricles_data/csvs/ct_splits_df.csv", index=False)

### [Ignore] create atlas metadata

In [7]:
# output_path = data_paths.ATLAS_OUTPUT

# atlas_files = get_files(output_path, extensions=['.pt'])

# atlas_file_suids = np.unique([o.name.split("_")[0] for o in atlas_files])

# atlas_splitsdf = pd.DataFrame({"StudyInstanceId":atlas_file_suids, "SplitType":"train"})

# atlas_splitsdf.SplitType.value_counts()

# atlas_splitsdf.to_csv("/home/turgutluk/data/ventricles_data/csvs/atlas_splits_df.csv", index=False)

### MR CT DataSource 

- notl_brain_mr - [norm. image + brain mask]
- notl_brain_ct - [norm. image + brain mask]
- notl_ventricle_mr - [norm. skull stripped image + ventricles mask]
- notl_ventricle_ct - [norm. skull stripped image + ventricles mask]

In [26]:
from local.dicom2tensor import _plot_voxel

In [29]:
#export 
import yaml
with open(os.environ.get('YAML_DATA', '../data.yaml')) as f: data_config = yaml.load(f.read(), yaml.FullLoader)

# define input and output paths
input_paths = types.SimpleNamespace(
    ATLAS_PATH=data_config['input']['ATLAS_PATH'],
    MR_PATH=data_config['input']['MR_PATH'],
    CT_PATH=data_config['input']['CT_PATH'],
    MR_TEST2_PATH=data_config['input']['MR_TEST2_PATH'],
    CT_TEST2_PATH=data_config['input']['CT_TEST2_PATH'],
)

output_paths = types.SimpleNamespace(
    ATLAS=data_config['output']['ATLAS'],
    MR=data_config['output']['MR'],
    CT=data_config['output']['CT'])

csv_split_paths = types.SimpleNamespace(
    ATLAS=data_config['csv_splits']['ATLAS'],
    MR=data_config['csv_splits']['MR'],
    CT=data_config['csv_splits']['CT'])

In [31]:
#export
def tfm_image_pt(o, data_path): return data_path/(o + "_image_normalized.pt")
def tfm_brain_mask_pt(o, data_path): return data_path/(o + "_brain_mask.pt")
def tfm_skull_stripped_image_pt(o, data_path): return data_path/(o + "_skull_stripped_image_normalized.pt")
def tfm_ventricles_mask_pt(o, data_path): return data_path/(o + "_ventricles_mask.pt")
def unsqz(x): return x[None,...]

In [32]:
#export
def create_mr_ct_dsource(data_path, csv_splits_path, tfm_pt_x, tfm_pt_y):
    """
    data_path: directory of tensor files 
    csv_splits_path: csv path for split info
    tfm_pt_x: tfm_image_pt or tfm_skull_stripped_image_pt
    tfm_pt_y: tfm_brain_mask_pt or tfm_ventricles_mask_pt
    """
    data_path = Path(data_path)
    splits_df = pd.read_csv(csv_splits_path)
    suids = splits_df['StudyInstanceId'].values
    
    tfmx = [partial(tfm_pt_x, data_path=data_path), torch.load, unsqz]
    tfmy = [partial(tfm_pt_y, data_path=data_path), torch.load]
    
    train_suids = splits_df.loc[splits_df.SplitType == "train", "StudyInstanceId"].values # subset 0
    valid_suids = splits_df.loc[splits_df.SplitType == "valid", "StudyInstanceId"].values # subset 1
    test1_suids = splits_df.loc[splits_df.SplitType == "test1", "StudyInstanceId"].values # subset 2
    test2_suids = splits_df.loc[splits_df.SplitType == "test2", "StudyInstanceId"].values # subset 3
    
    train_idxs = [idx for idx, suid in enumerate(suids) if suid in train_suids] 
    valid_idxs = [idx for idx, suid in enumerate(suids) if suid in valid_suids] 
    test1_idxs = [idx for idx, suid in enumerate(suids) if suid in test1_suids] 
    test2_idxs = [idx for idx, suid in enumerate(suids) if suid in test2_suids] 
    
    dsource = DataSource(suids, tfms=[tfmx, tfmy], splits=[train_idxs, valid_idxs, test1_idxs, test2_idxs])
    return dsource

In [33]:
#export
get_notl_brain_mr_dsource = partial(create_mr_ct_dsource, output_paths.MR,
                                             csv_split_paths.MR,
                                             tfm_image_pt,
                                             tfm_brain_mask_pt)

get_notl_brain_ct_dsource = partial(create_mr_ct_dsource, output_paths.CT,
                                             csv_split_paths.CT,
                                             tfm_image_pt,
                                             tfm_brain_mask_pt)

get_notl_ventricle_mr_dsource = partial(create_mr_ct_dsource, output_paths.MR,
                                                 csv_split_paths.MR,
                                                 tfm_skull_stripped_image_pt,
                                                 tfm_ventricles_mask_pt)

get_notl_ventricle_ct_dsource = partial(create_mr_ct_dsource, output_paths.CT,
                                                 csv_split_paths.CT,
                                                 tfm_skull_stripped_image_pt,
                                                 tfm_ventricles_mask_pt)

In [34]:
#export
datasource_dict = {}
datasource_dict['notl_brain_mr'] = get_notl_brain_mr_dsource
datasource_dict['notl_brain_ct'] = get_notl_brain_ct_dsource
datasource_dict['notl_ventricle_mr'] = get_notl_ventricle_mr_dsource
datasource_dict['notl_ventricle_ct']  = get_notl_ventricle_ct_dsource

In [35]:
dsource = datasource_dict['notl_brain_mr'](); len(dsource.splits) # train, valid, test1, test2

4

### Atlas DataSource

- atlas_ventricle_mr - [norm. skull stripped image + ventricles mask]
- atlas_brain_mr - [norm. image + brain mask]

In [36]:
#export
def atlas_tfm_image_pt(o): return o.parent/(o.name + "_image_normalized.pt")
def atlas_tfm_brain_mask_pt(o): return o.parent/(o.name + "_brain_mask.pt")
def atlas_tfm_skull_stripped_image_pt(o): return o.parent/(o.name + "_skull_stripped_image_normalized.pt")
def atlas_tfm_ventricles_mask_pt(o): return o.parent/(o.name + "_ventricles_mask.pt")

In [37]:
#export
def create_atlas_dsource(atlas_path, mr_path, atlas_splits_path, mr_splits_path, tfm_pt_x, tfm_pt_y):
    """
 
    tfm_pt_x: tfm_image_pt or tfm_skull_stripped_image_pt
    tfm_pt_y: tfm_brain_mask_pt or tfm_ventricles_mask_pt
    """
    atlas_path = Path(atlas_path)
    mr_path = Path(mr_path)
    atlas_splits_df = pd.read_csv(atlas_splits_path)
    mr_splits_df = pd.read_csv(mr_splits_path)
    
    tfmx = [tfm_pt_x, torch.load, unsqz]
    tfmy = [tfm_pt_y, torch.load]
    
    train_suids = atlas_splits_df.loc[atlas_splits_df.SplitType == "train", "StudyInstanceId"].values # subset 0
    valid_suids = mr_splits_df.loc[mr_splits_df.SplitType == "valid", "StudyInstanceId"].values # subset 1
    
    train_items, valid_items = [atlas_path/o for o in train_suids], [mr_path/o for o in valid_suids]
    items = train_items + valid_items
    train_idxs = np.arange(len(train_items))
    valid_idxs = len(train_idxs) + np.arange(len(valid_items))

    dsource = DataSource(items, tfms=[tfmx, tfmy], splits=[train_idxs, valid_idxs])
    return dsource

In [38]:
#export
get_atlas_brain_mr_dsource = partial(create_atlas_dsource, output_paths.ATLAS,
                                                  output_paths.MR,
                                                  csv_split_paths.ATLAS,
                                                  csv_split_paths.MR,
                                                  atlas_tfm_image_pt,
                                                  atlas_tfm_brain_mask_pt)

get_atlas_ventricle_mr_dsource = partial(create_atlas_dsource, output_paths.ATLAS,
                                                  output_paths.MR,
                                                  csv_split_paths.ATLAS,
                                                  csv_split_paths.MR,
                                                  atlas_tfm_skull_stripped_image_pt,
                                                  atlas_tfm_ventricles_mask_pt)

In [39]:
#export
datasource_dict['atlas_brain_mr'] = get_atlas_ventricle_mr_dsource
datasource_dict['atlas_ventricle_mr'] = get_atlas_brain_mr_dsource

In [40]:
dsource = datasource_dict['atlas_ventricle_mr'](); len(dsource.splits) # only train and valid

2

### Export

In [42]:
from local.notebook.export import notebook2script
notebook2script("2) datasource.ipynb")

Converted 2) datasource.ipynb.


### fin